# AMOCarray demo

The purpose of this notebook is to demonstrate the functionality of `AMOCarray`.

The demo is organised to show

- Step 1: Loading and plotting a sample dataset

- Step 2: Exploring the dataset attributes and variables.

Note that when you submit a pull request, you should `clear all outputs` from your python notebook for a cleaner merge.


In [ ]:
import pathlib
import sys

script_dir = pathlib.Path().parent.absolute()
parent_dir = script_dir.parents[0]
sys.path.append(str(parent_dir))

import importlib

import xarray as xr
import os
from amocatlas import readers, plotters, standardise, utilities

In [ ]:
# Specify the path for writing datafiles
data_path = os.path.join(parent_dir, "data")

### Load RAPID 26°N

In [ ]:
# Load data from data/moc_transports (Quick start)
ds_rapid = readers.load_sample_dataset()
ds_rapid = standardise.standardise_rapid(ds_rapid, ds_rapid.attrs["source_file"])

# Load data from data/moc_transports (Full dataset)
datasetsRAPID = readers.load_dataset("rapid", transport_only=True)
standardRAPID = [
    standardise.standardise_rapid(ds, ds.attrs["source_file"]) for ds in datasetsRAPID
]



In [ ]:
# Plot RAPID timeseries

plotters.plot_amoc_timeseries(
    data=[standardRAPID[0]],
    varnames=["moc_mar_hc10"],
    labels=[""],
    resample_monthly=True,
    plot_raw=True,
    title="RAPID 26°N"
)

### Load MOVE 16°N

In [ ]:
datasetsMOVE = readers.load_dataset("move")
standardMOVE = [
    standardise.standardise_move(ds, ds.attrs["source_file"]) for ds in datasetsMOVE
]

In [ ]:
# Plot MOVE timeseries

plotters.plot_amoc_timeseries(
    data=[standardMOVE[0]],
    varnames=["TRANSPORT_TOTAL"],
    labels=[""],
    colors=["darkgreen"],
    resample_monthly=True,
    plot_raw=True,
    title="MOVE 16°N - NADW Transport"
)

### Load OSNAP

In [ ]:
datasetsOSNAP = readers.load_dataset("osnap", transport_only = True)
standardOSNAP = [
    standardise.standardise_osnap(ds, ds.attrs["source_file"]) for ds in datasetsOSNAP
]


In [ ]:
# Plot OSNAP timeseries

plotters.plot_amoc_timeseries(
    data=[standardOSNAP[0]],
    varnames=["MOC_ALL"],
    labels=[""],
    colors=["darkblue"],
    resample_monthly=True,
    plot_raw=True,
    title="OSNAP"
)

### Load SAMBA 34.5°S

In [ ]:
datasetsSAMBA = readers.load_dataset("SAMBA")
standardSAMBA = [
    standardise.standardise_samba(ds, ds.attrs["source_file"]) for ds in datasetsSAMBA
]

In [ ]:
# Plot SAMBA timeseries

plotters.plot_amoc_timeseries(
    data=[standardSAMBA[0], standardSAMBA[1]],
    varnames=["UPPER_TRANSPORT", "MOC"],
    labels=["Kersale et al. 2020", "Meinen et al. 2018"],
    colors=["grey", "blue"],
    title="SAMBA 34.5°S",
    time_limits=("2000-01-01", "2022-12-31"),
    ylim=(-25, 25),
    resample_monthly=True,
    plot_raw=False # Raw data is a little spiky
)

###  Load FW2015

In [ ]:
datasetsfw2015 = readers.load_dataset("fw2015")
standardfw2015 = [
    standardise.standardise_fw2015(ds, ds.attrs["source_file"]) for ds in datasetsfw2015
]
plotters.show_variables(standardfw2015[0])

In [ ]:
# Plot timeseries

plotters.plot_amoc_timeseries(
    data=[standardfw2015[0]],
    varnames=["MOC_PROXY"],
    labels=[""],
    colors=["darkblue"],
    resample_monthly=True,
    plot_raw=True,
    title="FW2015"
)

### LOAD MOCHA 26.5°N

In [ ]:
datasetsMOCHA = readers.load_dataset("mocha")
standardMOCHA = [
    standardise.standardise_mocha(ds, ds.attrs["source_file"]) for ds in datasetsMOCHA
]

plotters.show_variables(standardMOCHA[0])

In [ ]:
# Plot timeseries

fig, ax = plotters.plot_amoc_timeseries(
    data=[standardMOCHA[0],standardMOCHA[0],standardMOCHA[0]],
    varnames=["Q_ot","Q_gyre","Q_sum"],
    labels=["Overturning","Gyre","Sum"],
    colors=["red","darkblue","black"],
    resample_monthly=True,
    plot_raw=False,
    title="MOCHA"
)
ax.legend(loc="lower right")


### LOAD 41°N

In [ ]:
datasets41n = readers.load_dataset("41n", transport_only=False)
standard41n = [
    standardise.standardise_41n(ds, ds.attrs["source_file"]) for ds in datasets41n
]

In [ ]:
plotters.plot_amoc_timeseries(
    data=[standard41n[0]],
    varnames=["MOT"],
    labels=[""],
    resample_monthly=True,
    plot_raw=False,
    colors=["darkblue"],
    title="41N"
)

### LOAD DSO

In [ ]:
datasetsDSO = readers.load_dataset("DSO", transport_only=False)
standardDSO = [
    standardise.standardise_dso(ds, ds.attrs["source_file"]) for ds in datasetsDSO
]

In [ ]:
plotters.plot_amoc_timeseries(
    data=[standardDSO[0]],
    varnames=["DSO"],
    labels=[""],
    resample_monthly=True,
    plot_raw=True,
    colors=["yellow"],
    title="DSO"
)

### Monthly Anomalies Overview

In [ ]:
plotters.plot_monthly_anomalies(
    osnap_data=standardOSNAP[0]["MOC_ALL"],
    rapid_data=standardRAPID[0]["moc_mar_hc10"],
    move_data=-standardMOVE[0]["TRANSPORT_TOTAL"],
    samba_data=standardSAMBA[1]["MOC"],
    fw2015_data=standardfw2015[0]["MOC_PROXY"],
    fortyone_data = standard41n[0]["MOT"],
    dso_data = standardDSO[0]["DSO"],
    osnap_label="OSNAP",
    rapid_label="RAPID 26°N",
    move_label="MOVE 16°N",
    samba_label="SAMBA 34.5°S",
    fw2015_label="FW2015",
    fortyone_label = "41°N",
    dso_label = "DS Overflow Transport"
)

## Other components

It is also possible to manipulate (filter) and plot other components of the AMOC.

In [ ]:
clim = standardRAPID[0].groupby("TIME.month").mean("TIME")
tmp = standardRAPID[0].groupby("TIME.month") - clim
filtRAPID = tmp.rolling(TIME = 500, center = True).mean()

fig,ax = plotters.plot_amoc_timeseries(
    data=[filtRAPID],
    varnames=["t_ld10"],
    labels=[""],
    resample_monthly=True,
    plot_raw=True,
    title="RAPID 26°N - t_ld10"
)
ax.set_ylim(2, -2)

fig.show()
